In [1]:
from fastai.vision import *
%config IPCompleter.greedy=True

Download data from https://www.kaggle.com/seesee/siim-train-test

Put this into `data/245622-651264-bundle-archive.zip`

In [ ]:
!unzip ../data/245622-651264-bundle-archive.zip -d ../data/

In [46]:
files = Path("../data/siim/dicom-images-train").glob('**/*.dcm')
input_images = [path.relative_to("../data/siim/dicom-images-train") for path in files]

file_map = {path.stem : path for path in input_images}

In [47]:
def remap(value):
    if value == "-1":
        return "no"
    else:
        return "yes"

df = pd.read_csv('../data/siim/train-rle.csv', header='infer')
df["present"] = df[" EncodedPixels"].map(remap)
df["ImageId"] = df["ImageId"].map(lambda filename: file_map[filename])
df.head()

,ImageId,EncodedPixels,present
0,1.2.276.0.7230010.3.1.2.8323329.6904.151787520...,-1,no
1,1.2.276.0.7230010.3.1.2.8323329.13666.15178752...,557374 2 1015 8 1009 14 1002 20 997 26 990 32 ...,yes
2,1.2.276.0.7230010.3.1.2.8323329.11028.15178752...,-1,no
3,1.2.276.0.7230010.3.1.2.8323329.10366.15178752...,514175 10 1008 29 994 30 993 32 991 33 990 34 ...,yes
4,1.2.276.0.7230010.3.1.2.8323329.10016.15178752...,592184 33 976 58 956 73 941 88 926 102 917 109...,yes


In [ ]:
np.random.seed(42)
data = ImageDataBunch.from_df(Path('../data/siim/dicom-images-train'), df, suffix=".dcm", valid_pct=0.2,
        label_col=2,
        ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)